In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

     |████████████████████████████████| 93 kB 3.4 MB/s  eta 0:00:01


In [2]:
#hiden cell
CLIENT_ID = '**********************' # your Foursquare ID
CLIENT_SECRET = '*******************************' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: *******************
CLIENT_SECRET:******************


In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "Halal Restaurant",
    "categoryId":"52e81612bcbc57f1066b79ff",
    "near": "Tokyo",
    "radius": 100000,
    "limit": 1000}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [4]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [5]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Tokyo', 'Tokyo')

In [6]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 36.56520377577499, 'lng': 140.72021011146663},
  'sw': {'lat': 35.38956244600891, 'lng': 138.66551894707302}},
 82)

In [7]:
d["geocode"]

{'what': '',
 'where': 'tokyo',
 'center': {'lat': 35.6895, 'lng': 139.69171},
 'displayString': 'Tokyo, Tōkyō, Japan',
 'cc': 'JP',
 'geometry': {'bounds': {'ne': {'lat': 35.898700999201466,
    'lng': 139.91899196935927},
   'sw': {'lat': 35.501102112000744, 'lng': 138.9427000242654}}},
 'slug': 'tokyo',
 'longId': '72057594039778083'}

In [8]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [9]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [10]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 76


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4becbcc30ddaa59351071ec3',
  'name': 'Halal Wagyu Yakiniku PANGA (ハラール和牛焼肉ぱんが)',
  'location': {'address': '台東3-27-9',
   'lat': 35.70493086805107,
   'lng': 139.78107877702692,
   'labeledLatLngs': [{'label': 'display',
     'lat': 35.70493086805107,
     'lng': 139.78107877702692}],
   'postalCode': '110-0016',
   'cc': 'JP',
   'city': '台東区',
   'state': '東京都',
   'country': '日本',
   'formattedAddress': ['台東3-27-9', '台東区, 東京都', '110-0016', '日本']},
  'categories': [{'id': '52e81612bcbc57f1066b79ff',
    'name': 'Halal Restaurant',
    'pluralName': 'Halal Restaurants',
    'shortName': 'Halal',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/halal_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4becbcc30ddaa59351071ec3-0'}

In [11]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i Library" % len(df))
df.head(30)

{'address': '台東3-27-9', 'lat': 35.70493086805107, 'lng': 139.78107877702692, 'labeledLatLngs': [{'label': 'display', 'lat': 35.70493086805107, 'lng': 139.78107877702692}], 'postalCode': '110-0016', 'cc': 'JP', 'city': '台東区', 'state': '東京都', 'country': '日本', 'formattedAddress': ['台東3-27-9', '台東区, 東京都', '110-0016', '日本']}
{'address': '江東橋2-18-6', 'lat': 35.695927, 'lng': 139.81183, 'labeledLatLngs': [{'label': 'display', 'lat': 35.695927, 'lng': 139.81183}], 'postalCode': '130-0022', 'cc': 'JP', 'neighborhood': '錦糸町, 東京', 'city': '東京', 'state': '東京都', 'country': '日本', 'formattedAddress': ['江東橋2-18-6', '墨田区, 東京都', '130-0022', '日本']}
{'address': '神田松永町16', 'crossStreet': '内尾松永ビル 1F', 'lat': 35.699970486152, 'lng': 139.7754316895416, 'labeledLatLngs': [{'label': 'display', 'lat': 35.699970486152, 'lng': 139.7754316895416}], 'postalCode': '101-0023', 'cc': 'JP', 'neighborhood': '秋葉原', 'city': '東京', 'state': '東京都', 'country': '日本', 'formattedAddress': ['神田松永町16 (内尾松永ビル 1F)', '千代田区, 東京都', '101

,uid,name,shortname,address,postalcode,lat,lng
0,4becbcc30ddaa59351071ec3,Halal Wagyu Yakiniku PANGA (ハラール和牛焼肉ぱんが),Halal,,110-0016,35.704931,139.781079
1,548c263a498e280856c7c21a,Tokyo Chinese Muslim Restaurant (東京ムスリム飯店),Halal,,130-0022,35.695927,139.811830
2,59c8d79e4382ab16da21736e,CoCo Ichibanya Halal Akihabara (CoCo壱番屋 ハラール秋葉原店),Halal,,101-0023,35.699970,139.775432
3,53e23062498e17de4b769c95,Halima Kebab Biryani (ハリマ ケバブ ビリヤニ),Halal,,110-0015,35.711734,139.780322
4,4bbaa95053649c74755849fb,Salma Tikka & Biryani,Halal,,108-0074,35.625819,139.737571
5,59b4bef373fe25689b5f3f04,ザムザムの泉,Halal,,332-0021,35.812129,139.702851
6,529812ab11d22448b7dbe55d,NASCO FOOD COURT,Halal,,169-0073,35.701982,139.699808
7,52d8952211d2e13e873f22cd,Solti Khaja Ghar (ソルティカージャガル),Halal,,169-0073,35.702289,139.699888
8,5556dc93498e67b288b63c40,HALALラーメン 浅草成田屋,Halal,,111-0032,35.714517,139.794567
9,4c8b011b2e333704e569c941,Khazana (カザーナ),Halal,,220-0012,35.456851,139.633358


In [12]:
Tokyo_center = d["geocode"]["center"]
Tokyo_center

{'lat': 35.6895, 'lng': 139.69171}

In [13]:
from folium import plugins


map_tokyo= folium.Map(location=[35.6895,  139.69171], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=2,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_tokyo)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_tokyo.add_child(plugins.HeatMap(hm_data))

map_tokyo